In [ ]:
import math as m
import numpy as np
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from tqdm import tqdm
import torch

In [ ]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states(coeff, abstract = False):    # coeff list like [a0,a1,b0,b1]
    if abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [ ]:
def roundc(c, digits):
    if c.imag == 0:
        return round(c.real, digits)
    else:
        return round(c.real, digits) + round(c.imag, digits) * 1j

In [ ]:
# creating the SIC POVM matrices
w = torch.exp((2/3)*torch.complex(torch.tensor([0.0]), torch.tensor([1.0]))*m.pi)
POVM_vec = (1/(2**.5))*(torch.tensor([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]], dtype=torch.cdouble))  # an array of POVM direction vectors
POVM_elts = [(1/3)*torch.outer(torch.conj(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrix

In [ ]:
def torch_out(w0, f0, f1, a0, a1):

    y_pred = torch.empty(9)
    
    w1 = 1-w0
    b0 = torch.sqrt(1 - a0**2)
    b1 = torch.sqrt(1 - a1**2)

    n2 = torch.tensor([2.0])
    
    y_pred[0] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p1
    y_pred[1] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p2
    y_pred[2] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1)*(a1**2)*(b1**2))) #p3
    y_pred[3] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(3*f0 - 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(3*f1 - 2*torch.pi/3)*a1*b1)) #p4
    y_pred[4] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 4*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 4*torch.pi/3)*a1*b1)) #p5
    y_pred[5] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 + 2*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 + 2*torch.pi/3)*(a1**2)*(b1**2))) #p6
    y_pred[6] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(3*f0 - 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(3*f1 - 2*torch.pi/3)*a1*b1)) #p7
    y_pred[7] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 2*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 2*torch.pi/3)*a1*b1)) #p8
    y_pred[8] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 - 4*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 - 4*torch.pi/3)*(a1**2)*(b1**2))) #p9

    return y_pred

In [ ]:
def floss(ny, w0, f0, f1, a0,  a1):
    y_pred = torch_out(w0, f0, f1, a0, a1)
    log_likelyhood = -torch.sum(ny * torch.log(y_pred))
    return log_likelyhood

In [ ]:
def update_lr(lr, i):
    return lr/(i+1)

In [ ]:
def grad_fid(learning_rate,N, coeff, priors, POVM_elts):
    #locals().clear()
    initial_states = Creating_states(abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    
    psi0sq = torch.tensor(psi0sq, dtype=torch.float32)
    psi1sq = torch.tensor(psi1sq, dtype=torch.float32)

    psi0psi0 = [psi0sq[0], torch.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]
    psi1psi1 = [psi1sq[0], torch.sqrt(psi1sq[1]**2+psi1sq[2]**2), psi1sq[3]]  # creating square states

    vec_psi0psi0 = torch.tensor(psi0psi0, dtype=torch.cdouble)
    vec_psi1psi1 = torch.tensor(psi1psi1, dtype=torch.cdouble)

    rho = priors[0]*torch.outer(vec_psi0psi0, torch.conj(vec_psi0psi0)) + priors[1]*torch.outer(vec_psi1psi1, torch.conj(vec_psi1psi1))  # theoretical density matrix with priors 1/2 each.

    prob_vec =  [torch.trace(torch.matmul(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]

    ##################################### GRaDIENT DESCENT ##############################################
    
    nj_vec = torch.tensor(nj_vec, dtype = torch.int64)
    
    # Initialize parameters
    w0 = torch.empty(1).uniform_(0, 1).requires_grad_()
    f0 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_() #torch.tensor([0.0], requires_grad=True)
    f1 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_() #torch.tensor([0.0], requires_grad=True)
    a0 = torch.empty(1).uniform_(0, 1).requires_grad_()
    a1 = torch.empty(1).uniform_(0, 1).requires_grad_()

    '''w0 = torch.tensor([.55], requires_grad=True)
    f0 = torch.tensor([0.1], requires_grad=True)
    f1 = torch.tensor([0.1], requires_grad=True)
    a0 = torch.tensor([0.1], requires_grad=True)
    a1 = torch.tensor([0.9], requires_grad=True)'''

    # Set learning rate
    #learning_rate = 0.001 + 1**(N/10000) #0.01
    
    #print(f'exp learning rate: {N/10000}')
    #print(f'learning rate: {learning_rate}')

    # Create a list to store the loss at each epoch
    loss_history = []
    w0_grad = []
    f0_grad = []
    f1_grad = []
    a0_grad = []
    a1_grad = []


    # Train for N epochs
    for i in range(100):
        
        
        # Forward pass:

        loss = floss(nj_vec, w0, f0, f1, a0, a1)
        # append the current loss to the history
        loss_history.append(loss.item())
        

        # Backward pass: compute gradients
        loss.backward()

        w0_grad.append(w0.grad.item())
        f0_grad.append(f0.grad.item())
        f1_grad.append(f1.grad.item())
        a0_grad.append(a0.grad.item())
        a1_grad.append(a1.grad.item())
        
        # Update parameters using gradient descent
        with torch.no_grad():
            w0 -= learning_rate * w0.grad
            f0 -= learning_rate * f0.grad
            f1 -= learning_rate * f1.grad
            a0 -= learning_rate * a0.grad
            a1 -= learning_rate * a1.grad

        # Zero gradients for the next iteration
        w0.grad.zero_()
        f0.grad.zero_()
        f1.grad.zero_()
        a0.grad.zero_()
        a1.grad.zero_()

    # after the training loop
    plt.figure()
    plt.plot(loss_history, label='Loss')
    plt.plot(w0_grad, label='w0_grad')
    plt.plot(f0_grad, label='f0_grad')
    plt.plot(f1_grad, label='f1_grad')
    plt.plot(a0_grad, label='a0_grad')
    plt.plot(a1_grad, label='a1_grad')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()                                   
    
    w1 = 1 - w0
    b0 = torch.sqrt(1 - a0**2)
    b1 = torch.sqrt(1 - a1**2)
    ph0 = torch.cos(f0) + 1j*torch.sin(f0)
    ph1 = torch.cos(f1) + 1j*torch.sin(f1)
    
    psi0n = torch.tensor([a0, ph0*a1], dtype=torch.cdouble)
    psi1n = torch.tensor([b0, ph1*b1], dtype=torch.cdouble)
    psi0t = torch.tensor(initial_states[0], dtype=torch.cdouble)
    psi1t = torch.tensor(initial_states[1], dtype=torch.cdouble)

    fid0 = torch.abs(torch.dot(psi0n, torch.conj(psi0t)))**2
    fid1 = torch.abs(torch.dot(psi1n, torch.conj(psi1t)))**2

    fid = [fid0.item(), fid1.item()]
    p = [w0.item(), w1.item()]

    return([fid,p])

In [ ]:
'''coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

#learning_rate = 0.001

initial_states = Creating_states(abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]    # created the states to be discriminated

psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   

psi0sq = torch.tensor(psi0sq, dtype=torch.float32)
psi1sq = torch.tensor(psi1sq, dtype=torch.float32)

psi0psi0 = [psi0sq[0], torch.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]
psi1psi1 = [psi1sq[0], torch.sqrt(psi1sq[1]**2+psi1sq[2]**2), psi1sq[3]]  # creating square states

vec_psi0psi0 = torch.tensor(psi0psi0, dtype=torch.cdouble)
vec_psi1psi1 = torch.tensor(psi1psi1, dtype=torch.cdouble)

rho = priors[0]*torch.outer(vec_psi0psi0, torch.conj(vec_psi0psi0)) + priors[1]*torch.outer(vec_psi1psi1, torch.conj(vec_psi1psi1))  # theoretical density matrix with priors 1/2 each.

prob_vec =  [torch.trace(torch.matmul(POVM_elts[i],rho)) for i in range(9)] 
prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]

##################################### GRaDIENT DESCENT ##############################################
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for f0, f1, t0, t1
f0_fixed = torch.tensor([torch.pi / 2])
f1_fixed = torch.tensor([torch.pi / 4])
t0_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 4])

# Define the range of values for a
a_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of a
torch_out_values = [torch_out(a,f0_fixed,f1_fixed,t0_fixed,t1_fixed)[0] for a in a_values]
floss_values = [floss(nj_vec, a, f0_fixed, f1_fixed, t0_fixed, t1_fixed) for a in a_values]
#torch_out_values = [grad_fid(learning_rate, N, coeff, priors, POVM_elts)[0] for a in a_values]

# Plot the dependence of torch_out on a
plt.plot(a_values, floss_values)
#plt.plot(a_values.detach().numpy(), torch_out_values)
plt.xlabel('a')
plt.ylabel('floss')
plt.show()'''

In [ ]:
coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
trials = [100*(i+1) for i in range(100)]
#trials = [10000]
p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials]

lr0 = 0.001

initial_states = Creating_states(abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]

output_f = []
output_p = []

pfid  = [1 for i in trials]    # perfect fidelity of 1
for i,t in tqdm(enumerate(trials)):
    #print(f'i_N: {i}')
    lr = update_lr(lr0,i)
    #print(f'learning rate: {lr}')
    output_f.append(grad_fid(lr, t,coeff,priors,POVM_elts)[0])
    output_p.append(grad_fid(lr, t,coeff,priors,POVM_elts)[1])

#output_f = [grad_fid(lr0, i, coeff, priors, POVM_elts)[0] for i in tqdm(trials)]
#output_p = [grad_fid(lr0, i, coeff, priors, POVM_elts)[1] for i in tqdm(trials)]

outputf = list(map(list, zip(*output_f)))
outputp = list(map(list, zip(*output_p)))

#print(outputf)
#print(outputp)

In [ ]:
fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_ylim(0,1)
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_ylim(0,1)
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_ylim(0,1)
ax2c.set_ylim(0,1)
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()